In [1]:
from dotenv import load_dotenv

load_dotenv()

from openai import OpenAI

openai = OpenAI()


In [24]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List, Optional
#from neo4jconnect import Neo4jConnection

from aiohttp import ClientSession
from bs4 import BeautifulSoup


app = FastAPI(title="Nonprofit Recommendaation Agent", description="Search and summarize nonprofits and recommend nonprofits personalized to user prompts")

#neo4j_conn = Neo4jConnection(uri="neo4j://neo4j:7687", user="neo4j", password="secretpassword")

async def scrape(query: str) -> list: #doesn't work 
    async with ClientSession() as session:
        url = f"https://www.duckduckgo.com/search?q={query.replace(' ', '+')}+site:.org" #can remove the .org
        async with session.get(url) as response:
            html = await response.text()
            soup = BeautifulSoup(html, 'html.parser')

            links = []
            for link in soup.select('a'):
                href = link.get('href')
                if href and "url?q=" in href:  # Filter to find the actual URL
                    url = href.split("url?q=")[1].split("&")[0]
                    links.append(url)
            return links[:5]
        
async def generate_search_keywords(prompt: str) -> List[dict]:
    



In [5]:
import httpx
import openai
from aiohttp import ClientSession
from bs4 import BeautifulSoup
import pandas as pd

SERP_API_KEY = "1c92173370dc002dd7a0053eb47eeef87f9cce81bc61ccdcc766c1672a4b0087"

SEARCH_ASSISTANT = """
You are a non-profit expert and people ask you for help looking for the best educational non-profit for their needs.
Your job is to create a search query based on a user's prompt, that will find non-profit orginizations when
typed into a search engine.
"""

async def search_serpapi(prompt: str) -> list:
    print(f"searchapi function {prompt}")
    query = prompt
    url = f"https://serpapi.com/search.json?q={query}&engine=google&api_key={SERP_API_KEY}"
    async with httpx.AsyncClient() as client:
        response = await client.get(url)
        data = response.json()
        
        # Extract URLs and snippets from organic results
        results = [
            {
                "title": item["title"],
                "link": item["link"],
                "snippet": item.get("snippet", "")
            }
            for item in data.get("organic_results", [])[:5]
        ]
        
        return results

async def summarize_with_ai(text: str, url: str) -> str:
    response = openai.chat.completions.create(
        model='anthropic.claude-3.5-sonnet.v2',
        messages=[
            {
                'role': 'user',
                'content': f"Summarize this snippet about a non-profit:\n\n{text}\n\nInclude the context from this website: {url}"
            },
        ],
    )
    # Extract the content from the response
    summary = response.choices[0].message.content
    return summary.strip()

async def fetch_full_page(url: str) -> str:
    async with ClientSession() as session:
        async with session.get(url) as response:
            html = await response.text()
            soup = BeautifulSoup(html, 'html.parser')
            paragraphs = soup.find_all('p')
            text = ' '.join([p.get_text() for p in paragraphs if len(p.get_text()) > 50])
            return text[:3000].strip()
        
# Fetch top 5 summaries of websites similar to the prompt
async def full_search_summarize(prompt: str) -> list:
    print(f"Starting SerpAPI search for prompt: {prompt}")
    
    search_results = await search_serpapi(prompt)
    print(f"Ending SerpAPI search for prompt: {prompt}")
    print(search_results)

    summaries = []
    for result in search_results:
        # Await the coroutine so that we get the actual summary text
        snippet_summary = await summarize_with_ai(result["snippet"], result["link"])
        summaries.append({
            "title": result["title"],
            "link": result["link"],
            "summary": snippet_summary
        })
    return summaries

user_prompt_ex = "financial aid educational non profit"

async def generate_table_from_prompt(prompt: str):
    # Perform the full search and summarization
    search_results = await full_search_summarize(prompt)
    
    # Prepare data for the table
    data = [{"Link": result["link"], "Summary": result["summary"]} for result in search_results]
    
    # Create a DataFrame for better visualization
    df = pd.DataFrame(data)
    
    # Print the table
    print(df.to_string(index=False))

# Run the async function (this line assumes you're in an async-capable environment)
await generate_table_from_prompt(user_prompt_ex)


Starting SerpAPI search for prompt: financial aid educational non profit
searchapi function financial aid educational non profit
Ending SerpAPI search for prompt: financial aid educational non profit
[{'title': 'Understanding Types of Financial Aid', 'link': 'https://studentaid.gov/understand-aid/types', 'snippet': ''}, {'title': 'NASFAA.org Home', 'link': 'https://www.nasfaa.org/', 'snippet': 'NASFAA is a nonpartisan, nonprofit association that provides professional development and services for financial aid administrators.'}, {'title': 'Financial Aid', 'link': 'https://customprograms.worldstrides.com/financial-aid/', 'snippet': 'We offer $500 in international financial assistance (IFA) for students registered for WorldStrides university programs in the current and subsequent calendar ...'}, {'title': 'FES | A Foundation of Support', 'link': 'https://www.fes.org/', 'snippet': 'FES is a nonprofit foundation that provides support to nonprofit organizations so they can focus on fulfillin

''